In [1]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from utils_plotting import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import metpy
import numbers

In [2]:
data_location = 'data'
outlooks, pph, reports = read_datasets(data_location, 'labelled')

reading outlooks
reading pph
reading storm reports


In [29]:
category_dict = {
    'NONE' : -1,
    'TSTM': 0,
    'MRGL': 1,
    'SLGT': 2,
    'ENH': 3,
    'MDT': 4,
    'HIGH': 5
}


season_dict = {
    'Winter': 0,
    'Spring': 1,
    'Summer': 2,
    'Fall': 3
}

region_dict = {
    'NONE': -1,
    'West': 0,
    'Great Plains': 1,
    'Midwest': 2,
    'Northeast': 3,
    'South': 4
}

tor_dict = {
    'NONE': -1,
    'EFU': 0,
    '(E)F0': 1,
    '(E)F1': 2,
    '(E)F2': 3,
    '(E)F3': 4,
    '(E)F4': 5,
    '(E)F5': 6
}

cats = ['MAX_CAT', 'SEASON', 'REGION', 'TOR_F']
dicts = [category_dict, season_dict, region_dict, tor_dict]

In [27]:
# remove unneeded variables and x, y dimensions
data = pph.drop(['lat', 'lon', 'p_perfect_wind', 'p_perfect_sig_wind', 'p_perfect_hail', 'p_perfect_sig_hail', 'p_perfect_tor', 'p_perfect_sig_tor', 'RAMP_CAT', 'p_perfect_total', 'PPH_CAT', 'REPORT_NUM', 'WINDSP_CAT', 'HAILSZ_CAT'])


# make data numerical
def convert_strings_to_ints(data, conversion_dict):
    return np.vectorize(conversion_dict.get)(data)

for cat, dic in zip(cats, dicts):

    data[cat] = xr.apply_ufunc(
        convert_strings_to_ints,  # function to apply
        data[cat],      # input data
        kwargs={'conversion_dict': dic},  # additional arguments
        vectorize=True            # vectorize the function
    )

data

# normalize, and consolidate date

<xarray.Dataset>
Dimensions:     (time: 16071, x: 93, y: 65)
Coordinates:
  * time        (time) object '197901010000' '197901020000' ... '202212310000'
  * x           (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 88.0 89.0 90.0 91.0 92.0
  * y           (y) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 60.0 61.0 62.0 63.0 64.0
Data variables: (12/15)
    MAX_CAT     (time) int32 -1 -1 -1 -1 -1 -1 -1 -1 -1 ... 0 -1 -1 0 0 0 1 1 1
    RAMP_UP     (time) int32 ...
    RAMP_DOWN   (time) int32 ...
    SEASON      (time) int32 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    REGION      (time) int32 4 4 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 1 4 -1
    PPH_NUM     (time) float64 ...
    ...          ...
    TOR_F       (time) int32 -1 -1 -1 -1 -1 -1 -1 -1 ... -1 -1 -1 -1 -1 2 2 -1
    WINDSP_NUM  (time) float64 ...
    HAILSZ_NUM  (time) float64 ...
    NEIGH_NUM   (time) float64 ...
    RMSE_NUM    (time) float64 ...
    BS_NUM      (time) float64 ...
Attributes:
    title:         Practically Perfect Wind Hindcasts
    grid:          80-km NCEP 211
    sigma:         1.5
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

In [28]:
# do pca

<xarray.Dataset>
Dimensions:             (time: 16071, x: 93, y: 65)
Coordinates:
  * time                (time) object '197901010000' ... '202212310000'
  * x                   (x) float64 0.0 1.0 2.0 3.0 4.0 ... 89.0 90.0 91.0 92.0
  * y                   (y) float64 0.0 1.0 2.0 3.0 4.0 ... 61.0 62.0 63.0 64.0
Data variables: (12/29)
    lat                 (y, x) float64 ...
    lon                 (y, x) float64 ...
    p_perfect_wind      (time, y, x) float64 ...
    p_perfect_sig_wind  (time, y, x) float64 ...
    p_perfect_hail      (time, y, x) float64 ...
    p_perfect_sig_hail  (time, y, x) float64 ...
    ...                  ...
    WINDSP_NUM          (time) float64 ...
    HAILSZ_CAT          (time) object ...
    HAILSZ_NUM          (time) float64 ...
    NEIGH_NUM           (time) float64 ...
    RMSE_NUM            (time) float64 ...
    BS_NUM              (time) float64 ...
Attributes:
    title:         Practically Perfect Wind Hindcasts
    grid:          80-km NCEP 211
    sigma:         1.5
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

In [ ]:
# cluster with, k-means, knn, optics, birch, dbscan look here for use cases https://scikit-learn.org/stable/modules/clustering.html